# Natural Language Processing with Disaster Tweets

The task was to classify whether tweets concern a natural disaster or not. The dataset comes from https://www.kaggle.com/competitions/nlp-getting-started

# Dataset overview

First things first - let's take a loot at the dataset to find out what we have to deal with.

In [104]:
import pandas as pd

train_df = pd.read_csv('train.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


The dataset consists of 4 columns, one of which being redundant (id), and the target. One can easily see that `keyword` and `location` are likely to be null.

In [105]:
print(f'''
Rows in total: {len(train_df)}
Rows with no keyword: {len(train_df[train_df['keyword'].isna()])}
Rows with no location: {len(train_df[train_df['location'].isna()])}
Rows with no text: {len(train_df[train_df['text'].isna()])}
''')


Rows in total: 7613
Rows with no keyword: 61
Rows with no location: 2533
Rows with no text: 0



Fortunately keyword is missing in only 61 rows and the location is present in most of the samples.

# Initial approach - Random Forest

We decided to start off with something simple and try to find an efficient solution that would not involve neural networks. 

## Preprocessing

As classifiers only operate on numeric values, the first challenge was to convert our text input into valid input. Having no prior experience with NLP, we focused mostly on creating metadata related columns.

In [106]:
import pandas as pd
from statistics import mean
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('vader_lexicon')

def count_capital_letters_part(text):
    capital_letters = 0
    c: str
    for c in text:
        if c.isupper():
            capital_letters += 1

    return capital_letters / len(text) if len(text) > 0 else 0
    
def avg_word_length(text):
    return mean([len(word) for word in text.split()])

def calc_compound_sentiment(text):
    if len(text) == 0:
        return 0
    
    return SentimentIntensityAnalyzer().polarity_scores(text)['compound']


def calc_location_credibility(loc):
    words = loc.split()

    if len(words) == 0:
        return 0

    score = 0
    for word in words:
        if word[0].isupper():
            score += 1
        else:
            score -= 1
    
    return score / len(words)

def calculate_top_keywords(df, y):
    top_target_keywords = df[y==1].keyword.value_counts().head(100)
    top_non_target_keywords = df[y==0].keyword.value_counts().head(100)

    words_1 = set(list(top_target_keywords.index))
    words_0 = set(list(top_non_target_keywords.index))
    intersection = list(words_1.intersection(words_0)) # identification of ambigual keywords

    top_target_keywords = top_target_keywords.drop(intersection)
    top_non_target_keywords = top_non_target_keywords.drop(intersection)
    
    return top_target_keywords, top_non_target_keywords


def generate_columns(df):
    _df = df.copy()
    generated_df = pd.DataFrame()
    _df[['keyword', 'location']] = _df[['keyword', 'location']].fillna(value='')
    _df['keyword'] = _df['keyword'].str.replace('%20', ' ') # %20 is actually a space but we don't want it
    
    top_target_keywords, top_non_target_keywords = calculate_top_keywords(X_train, y_train)
    
    generated_df['text_length'] = _df['text'].str.len()
    generated_df['capital_letters_part'] = _df['text'].apply(count_capital_letters_part)
    generated_df['avg_word_length'] = _df['text'].apply(avg_word_length)
    generated_df['sentiment'] = _df['text'].apply(calc_compound_sentiment)
    generated_df['hashtag_count'] = _df['text'].apply(lambda text: text.count('#'))
    generated_df['punctuation_marks_per_char'] = _df['text'].apply(lambda text: (text.count('?') + text.count('!') + text.count('.')) / len(text))
    generated_df['keyword_sentiment'] = _df['keyword'].apply(calc_compound_sentiment)
    generated_df['location_credibility'] = _df['location'].apply(calc_location_credibility)
    generated_df['target_keyword'] = _df['keyword'].apply(lambda keyword: keyword in top_target_keywords)
    generated_df['non_target_keyword'] = _df['keyword'].apply(lambda keyword: keyword in top_non_target_keywords)

    return generated_df



[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jakublange/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/jakublange/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


The most important of the functions above is the last one, which does the actual job of creating a new dataframe with numeric values based on the source dataframe.
Most of the columns are self-explanatory, except for a few of them:
- keyword_sentiment - overall ("compound") sentiment of text calculated using `SentimentIntensityAnalyzer` from `nltk` package
- location_credibility - this is a factor based on an observation, that strings in which each word starts with a capital letter are more likely to be a real location
- target_keyword - tells whether the keyword if the sample is among the top 100 keywords for samples with target equal to 1
- non_target_keyword - same as the previous one but for samples with target equal to 0

## Standardization

In [107]:
from sklearn.preprocessing import StandardScaler

def standardize(X):
  scaler = StandardScaler()
  return scaler.fit_transform(X)

## Preprocess function

In [108]:
def preprocess(X):
    _X = X.copy()
    _X = _X.iloc[:, 1:] # we don't want to include id as it should affect predictions
    _X = generate_columns(_X)
    _X = standardize(_X)
    return _X

## Building the model

In [109]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv('train.csv')
X = df.iloc[:, :-1] 
y = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                stratify=y, 
                                                test_size=0.3)

X_train_preprocessed = preprocess(X_train)

model = RandomForestClassifier()
model.fit(X_train_preprocessed, y_train)

RandomForestClassifier()

## Prediction

In [110]:
from sklearn.metrics import classification_report

X_test_preprocessed = preprocess(X_test)
y_pred = model.predict(X_test_preprocessed)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.82      0.78      1303
           1       0.72      0.62      0.67       981

    accuracy                           0.74      2284
   macro avg       0.73      0.72      0.73      2284
weighted avg       0.73      0.74      0.73      2284



# Second approach - Tfidf

In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS


def tfidf(column: pd.Series) -> tuple[TfidfVectorizer, pd.DataFrame]:
    vectorizer = TfidfVectorizer(token_pattern=r'(?u)\b[A-Za-z]+\b', # exclude digits
                                 stop_words=list(ENGLISH_STOP_WORDS)) # exclude stop words
    sparse_matrix = vectorizer.fit_transform(column.str.lower())
    feature_names = vectorizer.get_feature_names_out()
    
    return vectorizer, pd.DataFrame.sparse.from_spmatrix(sparse_matrix, columns=feature_names)

def generate_tfidf_columns(df) -> tuple[pd.DataFrame, TfidfVectorizer]:
    _df = df.copy()
    text_vectorizer, text_tfidf = tfidf(_df['text'])
    
    return text_tfidf, text_vectorizer

def apply_vectorizer_to_test(X_test: pd.DataFrame, text_vectorizer) -> pd.DataFrame:   
    vectorized_text = text_vectorizer.transform(X_test['text'])
    X_text = pd.DataFrame.sparse.from_spmatrix(vectorized_text, 
                                               columns=text_vectorizer.get_feature_names_out())
    
    return X_text

In [112]:
X_train_tfidf, text_vectorizer = generate_tfidf_columns(X_train)

tfidf_classifier = RandomForestClassifier()

tfidf_classifier.fit(X_train_tfidf, y_train)

RandomForestClassifier()

In [113]:
X_test_tfidf = apply_vectorizer_to_test(X_test, text_vectorizer)
y_pred_tfidf = tfidf_classifier.predict(X_test_tfidf)

In [114]:
print(classification_report(y_test, y_pred_tfidf))

              precision    recall  f1-score   support

           0       0.75      0.93      0.83      1303
           1       0.86      0.59      0.70       981

    accuracy                           0.78      2284
   macro avg       0.80      0.76      0.77      2284
weighted avg       0.80      0.78      0.77      2284



Tfidf seemes to be slightly more accurate than the previous approach.